In [2]:
# work in progress

In [3]:
import sys
from pathlib import Path
PROJECT_DIR = Path.cwd()
if PROJECT_DIR.stem == 'notebooks':
    PROJECT_DIR = PROJECT_DIR.parent
    sys.path.insert(0, '..')

INTERACTIVE_PLOTTING = True

# Overview

In [4]:
# NOTE
# Calibration consists of multiple sections pertaining:
#   - parameter estimation/validation 
#   - experimental instructions

# Pain threshold estimation
# Section 1: Preexposure to thermal stimuli
#   - familiarize the subject with the thermal stimuli
#   - binary responses to thermal stimuli
#   - Q: what kind of binary response? -> only asked at the end, if painful, mean is higher
#   - how many trials? how many temperatures? do we use the results?
# Section 2: Probabilistic pain threshold detection
#   - Bayesian adaptive design / sequential Bayesian design to:
#        1. update belief about the heat pain threshold and
#        2. decide on the temperature for the next trial.
#   - binary responses
#   - DONE

# Purely didactic for rating training
# Peri- or suprathreshold calibration
# Section 3: Scale translation
#   - didactic trials to support the transition from binary responses (Sections 1&2) to VAS responses
#   - purely instructional, to be done in psychopy
#   - Q: do I need this
# Section 4: Rating training
#   - Low intensities are applied exclusively to familiarize the subject with the visuomotor demands of the VAS scale
#   - Q: do I need this?

# Section 5: Psychometric-perceptual scaling
#   - (Section 5, includes all steps up to Section 6)
#   - Fixed temperature increments to familiarize subject with possible perceptual range
#   - Fixed intensity target regression: Perithresholding with fixed temperature increments, e.g. +0.5 °C, +1 °C
#   - Fixed VAS target regression: Perithresholding with fixed VAS targets, e.g. inferred temperatures for 30 VAS, 60 VAS
#   - Adaptive VAS target regression: Perithresholding to elaborate poorly sampled areas of the VAS range
#   - Q: How does this work? What is the difference between fixed intensity target regression and fixed VAS target regression?


# Section 6: Calibration validation
#   - apply results from Section 5 for a number of trials, e.g. to assess robustness

In [5]:
# TODO
# - ask andreas for values

In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

## Section 1: Preexposure to thermal stimuli

In [6]:
# Values from matlab setting script:

# % Preexposure (e.g. CalibHeat Section 1)
# P.calib.heat(n).preexposure.sInitCross  = 5; % forced wait after instructions/transitions
# P.calib.heat(n).preexposure.sInitCue    = 2; % in addition to sInitCross
# P.calib.heat(n).preexposure.intensities = [42 42.5 43 43.5]; % vector of intensities used for preexposure, intended to lead to binary decision "were any of these painful y/n"
# P.calib.heat(n).preexposure.sPlateau    = 10; 
# P.calib.heat(n).preexposure.sMinMaxITIs = [5 5]; 
# P.calib.heat(n).preexposure.sMinMaxCues = [2 2]; % in addition to sMinMaxITIs                 


In [7]:
preexposure_temperatures = [33, 34, 35, 36]
response = input(f'Were any of the preexposure stimuli ({str(preexposure_temperatures)} °C) painful? (y/n) ')
if response == "y":
    mean_temperature = 37
else:
    mean_temperature = 39

## Section 2: Probabilistic pain threshold estimation

In [8]:
# Step 1: Define a prior probability distribution for the heat pain threshold
min_temperature = mean_temperature - 6
max_temperature = mean_temperature + 6
range_temperature = np.arange(min_temperature, max_temperature+0.1, 0.1)
std_temperature = 3.5

likelihood_std = 1
reduction_factor = 0.95

prior = stats.norm.pdf(
    range_temperature, 
    loc=mean_temperature, 
    scale=std_temperature)
prior /= np.sum(prior)  # normalize to make it a valid probability distribution

# Step 2: Start the experiment with a predetermined temperature
current_temperature = np.round(
    range_temperature[np.argmax(prior)],
    1) # = mean_temperature from prior

# Step 3-end: Repeat for each trial
temperatures = [current_temperature]
priors = []
likelihoods = []
posteriors = []
trials = 8

for trial in range(trials):
    # Collect the subject's response
    response = input(f'Is {current_temperature} °C painful? (y/n) ')

    # Define a cdf likelihood function based on the response
    if response == 'y':
        likelihood = 1 - stats.norm.cdf(
            range_temperature, 
            loc=current_temperature, 
            scale=likelihood_std)
    else:
        likelihood = stats.norm.cdf(
            range_temperature,
            loc=current_temperature, 
            scale=likelihood_std)

    # Decrease the standard deviation of the likelihood function as we gain more information
    likelihood_std *= reduction_factor

    # Update the prior distribution with the likelihood function to get a posterior distribution
    posterior = likelihood * prior
    posterior /= np.sum(posterior)  # normalize
    
    # Choose the temperature for the next trial based on the posterior distribution
    current_temperature = np.round(
        range_temperature[np.argmax(posterior)],
        1) # temperature with highest probability

    # Store the distributions and temperature
    priors.append(prior)
    likelihoods.append(likelihood)
    posteriors.append(posterior)
    temperatures.append(current_temperature)
    
    # Update the prior for the next iteration
    prior = np.copy(posterior)

print(f"{temperatures}\n{np.diff(temperatures)}")
vas_0 = temperatures[-1]

[38.0, 39.6, 40.8, 41.8, 42.7, 43.5, 44.3, 45.0, 45.0]
[1.6 1.2 1.  0.9 0.8 0.8 0.7 0. ]


In [17]:
if INTERACTIVE_PLOTTING:
    # reloade vs window if plot appears twice
    import ipywidgets as widgets 

    def plot_trial(trial):
        plt.clf()
        plt.plot(range_temperature, priors[trial], label='Prior')
        plt.plot(range_temperature, likelihoods[trial], label='Likelihood')
        plt.plot(range_temperature, posteriors[trial], label='Posterior')
        plt.title(f'Trial {trial+1}')
        plt.xlim([min_temperature, max_temperature])
        plt.ylim([0, 1])
        plt.xlabel('Temperature (°C)')
        plt.ylabel('Probability')
        plt.xticks(np.arange(min_temperature, max_temperature+1, 1))
        plt.legend(bbox_to_anchor=(0.5, -0.15), loc='upper center', ncol=3)

    # Create a slider for the trial number
    trial_slider = widgets.IntSlider(min=0, max=trials-1, step=1, value=0);

    # Create Next and Previous buttons, define click events and link them to the buttons
    next_button = widgets.Button(description="Next")
    prev_button = widgets.Button(description="Previous")
    def next_button_clicked(b): # b is the button instance
        if trial_slider.value < trial_slider.max:
            trial_slider.value += 1
    def prev_button_clicked(b):
        if trial_slider.value > trial_slider.min:
            trial_slider.value -= 1
    next_button.on_click(next_button_clicked)
    prev_button.on_click(prev_button_clicked)

    # Use the interact function to automatically update the plot when the slider is moved
    out = widgets.interactive_output(plot_trial, {'trial': trial_slider});

    # Display the slider and the buttons on top of the figure (box on box)
    widgets.VBox([widgets.HBox([trial_slider, prev_button, next_button]), out])

else:
    # One figure with all trials
    fig, ax = plt.subplots(trials, 1, figsize=(5, 10))
    for trial in range(trials):
        ax[trial].plot(range_temperature, priors[trial], label='Prior')
        ax[trial].plot(range_temperature, likelihoods[trial], label='Likelihood')
        ax[trial].plot(range_temperature, posteriors[trial], label='Posterior')
        ax[trial].set_title(f'Trial {trial+1}')
    plt.tight_layout()
    plt.show()

## Section 3-4: Rating training (purely instructional)

## Section 5: Psychometric-perceptual scaling

In [22]:
x = np.array([0, 1, 2, 3, 4, 5, 6, 7])
y = np.array([1, 2, 3, 4, 5, 3, 7, 8])

slope, intercept = np.polyfit(x, y, 1)
y_pred = slope * x + intercept

plt.scatter(x, y, label='Data')
plt.plot(x, y_pred, color='red', label='Fitted line')
plt.legend()
plt.show()

In [ ]:
response = input(f'How painful is it ({} °C) on a scale from 0 to 100?')


## Section 6: Calibration validation

In [13]:
import numpy as np
a = np.arange(1,10,0.1)
len(a) == a.shape[0]
a[90-1]

9.900000000000007